In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

SEED = 42
BATCH_SIZE = 8
MODEL = "v2"

if os.path.exists("./yolo"):
    shutil.rmtree("./yolo")

if not os.path.exists("./yolo/train"):
    os.makedirs("./yolo/train")
    
if not os.path.exists("./yolo/valid"):
    os.makedirs("./yolo/valid")
    
if not os.path.exists("./yolo/test"):
    os.makedirs("./yolo/test")    
    
if not os.path.exists("./results"):
    os.makedirs("./results")

def make_yolo_dataset(image_paths, txt_paths, type="train"):
    for image_path, txt_path in tqdm(zip(image_paths, txt_paths if not type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)        
        image_height, image_width, _ = source_image.shape
        
        target_image_path = f"./yolo/{type}/{os.path.basename(image_path)}"
        cv2.imwrite(target_image_path, source_image)
        
        if type == "test":
            continue
        
        with open(txt_path, "r") as reader:
            yolo_labels = []
            for line in reader.readlines():
                line = list(map(float, line.strip().split(" ")))
                class_name = int(line[0])
                x_min, y_min = float(min(line[5], line[7])), float(min(line[6], line[8]))
                x_max, y_max = float(max(line[1], line[3])), float(max(line[2], line[4]))
                x, y = float(((x_min + x_max) / 2) / image_width), float(((y_min + y_max) / 2) / image_height)
                w, h = abs(x_max - x_min) / image_width, abs(y_max - y_min) / image_height
                yolo_labels.append(f"{class_name} {x} {y} {w} {h}")
            
        target_label_txt = f"./yolo/{type}/{os.path.basename(txt_path)}"      
        with open(target_label_txt, "w") as writer:
            for yolo_label in yolo_labels:
                writer.write(f"{yolo_label}\n")

image_paths = sorted(glob("./data/train/*.png"))
txt_paths = sorted(glob("./data/train/*.txt"))

train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths = train_test_split(image_paths, txt_paths, test_size=0.1, random_state=SEED)

make_yolo_dataset(train_images_paths, train_txt_paths, "train")
make_yolo_dataset(valid_images_paths, valid_txt_paths, "valid")
make_yolo_dataset(sorted(glob("./data/test/*.png")), None, "test")

100%|██████████| 3400/3400 [08:19<00:00,  6.80it/s]


In [7]:
with open("./data/classes.txt", "r") as reader:
    lines = reader.readlines()
    classes = [line.strip().split(",")[1] for line in lines]

yaml_data = {
              "names": classes,
              "nc": len(classes),
              "path": r"C:\Users\wjswp\Downloads\Objectdetection\yolo",
              "train": "train",
              "val": "valid",
              "test": "test"
            }

with open("./yolo/custom.yaml", "w") as writer:
    yaml.dump(yaml_data, writer)

#model = YOLO(f"{MODEL}/train/weights/last.pt")
model = YOLO("yolov8n")
results = model.train(
    data="./yolo/custom.yaml",
    imgsz=(1024, 1024),
    epochs=200,
    batch=BATCH_SIZE,
    patience=5,
    workers=16,
    device=0,
    exist_ok=True,    
    project=f"{MODEL}",
    name="train",
    seed=SEED,
    pretrained=False,
    resume=False,
    optimizer="Adam",
    lr0=1e-3,
    augment=False,
    val=True,
    cache=True
    )

New https://pypi.org/project/ultralytics/8.0.114 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.113  Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11264MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=./yolo/custom.yaml, epochs=200, patience=5, batch=8, imgsz=(1024, 1024), save=True, save_period=-1, cache=True, device=0, workers=16, project=v2, name=train, exist_ok=True, pretrained=False, optimizer=Adam, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False,

In [23]:
def get_test_image_paths(test_image_paths):    
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

model = YOLO("v2/train/weights/best.pt")
test_image_paths = glob("./yolo/test/*.png")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=(1024, 1024), iou=0.2, conf=0.5, save_conf=True, save=False, save_txt=True, project=f"{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=False, verbose=False)
    if i % 5 == 0:
        clear_output(wait=True)

def yolo_to_labelme(line, image_width, image_height, txt_file_name):    
    file_name = txt_file_name.split("\\")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]
    
    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)
    
    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

infer_txts = glob(f"{MODEL}/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("\\")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"./yolo/test/{base_file_name}.png").shape[:2]        
    with open(infer_txt, "r") as reader:        
        lines = reader.readlines()        
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.to_csv(f"./{MODEL}.csv", index=False)

 99%|█████████▉| 421/425 [03:56<00:02,  1.81it/s]Results saved to v2\predict
3395 labels saved to v2\predict\labels
 99%|█████████▉| 422/425 [03:56<00:01,  1.81it/s]Results saved to v2\predict
3395 labels saved to v2\predict\labels
100%|█████████▉| 423/425 [03:57<00:01,  1.80it/s]Results saved to v2\predict
3395 labels saved to v2\predict\labels
100%|█████████▉| 424/425 [03:57<00:00,  1.83it/s]Results saved to v2\predict
3395 labels saved to v2\predict\labels
100%|██████████| 3395/3395 [02:24<00:00, 23.47it/s]


In [20]:
infer_txts = glob("v2/predict/labels/*.txt")
infer_txts[0].split('\\')[-1].split('.')[0]

'064442001'